# Pipelining with Titanic Data

### Data loading ...

In [180]:
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import fcluster, single, complete
from collections.abc import Iterable
import pandas as pd
import re
import numpy as np
train = pd.read_csv("./data/train.csv")
test = pd.read_csv("./data/test.csv")
combined = pd.concat([train, test], sort=False).reset_index(drop=True)

#### Some preprocessing of the data

In [187]:
def parse_name(s): 
    a, b = s.split(',')
    family_name = a.strip()
    title = b.split('.')[0].strip()
    first_name = b.split('.')[1].split()[0].strip()
    return (first_name.replace('(', '').replace(')', ''), title, family_name)

def parse_cabin_letter(column):
    letter_pat = re.compile('([A-Za-z])\d+')
    return list(map(lambda x: letter_pat.findall(str(x))[0] if letter_pat.findall(str(x)) else pd.np.nan, column.values.tolist()))

def parse_ticket_number(column):
    number_pat = re.compile('\d{3,}')
    numbers = map(lambda x: number_pat.findall(x)[0] if number_pat.findall(x) else pd.np.nan, column)
    return pd.Series(numbers)


def get_friendship_group(df):
    friendship_group_counter = 0
    if 'family_name' not in df.columns:
        family_names = pd.Series(map(lambda x: parse_name(x)[-1], df.Name))
    else:
        family_names = df.family_name
    cabins = pd.Series(map(parse_cabin_letter, df.Cabin))
    ticket_grouping = []
    for family, count in family_names.value_counts().items():
        family_mask = family_names == family
        
        if count == 1:
            ticket_grouping.append(friendship_group_counter)
            friendship_group_counter += 1
            continue

            
def get_family_name(df):
    family_names = pd.Series(map(lambda x: parse_name(x)[-1], df.Name))
    df_ = df.copy()
    df_.loc[:,'family_name'] = family_names
    return df_
            
def get_ticket_group(df):
    df_ = df.copy()
    grouped = df.Ticket.groupby(parse_ticket_number(df.Ticket))
    for ind, key in enumerate(grouped.indices):
        df_.loc[grouped.indices[key], 'ticket_group'] = ind
    return df_

def get_cabin_letter(df):
    df_ = df.copy()
    cabins = parse_cabin_letter(df.Cabin)
    df_.loc[:, 'cabin_na'] = pd.isnull(df.Cabin)
    df_.loc[:, 'cabin'] = cabins
    return df_

def get_is_alone(df):
    df_ = df.copy()
    df_.loc[:, 'is_alone'] = (df.loc[:, 'Parch'] + df.loc[:, 'SibSp'] + 1 == 1)
    return df_

def get_titles(df):
    df_ = df.copy()
    titles = pd.Series(map(lambda x: parse_name(x)[1], df.Name))
    df_.loc[:, 'title'] = titles
    return df_

def discretize_faries(df, ngroups=3):
    df_ = df.copy()
    df_.loc[:, 'fares'] = pd.cut(df_.loc[:,'Fare'], ngroups, labels=False)
    return df_

def estimate_age(df):
    estimates = []
    for ind, row in df.loc[df.Age.isnull(), :].iterrows():
        # NOTE: Could be rewritten using vectorized notation
        if row.title in ['Master', 'Mr', 'Miss', 'Rev', 'Dr']:
            estimates.append(df.groupby(['title', 'Sex']).median().loc[[row.title, row.Sex],'Age'].values[0])
        else:
            estimates.append(df.groupby(['Sex', 'Pclass']).median().loc[[row.Sex], 'Age'].values[0])
    df_ = df.copy()
    df_.loc[df.Age.isnull(), 'Age'] = estimates
    return df_


def get_cabin_groups(df):
    num_pat = re.compile('\d+')
    let_pat = re.compile('[a-zA-Z]')
    LONG_DISTANCE = 5
    MEDIUM_DISTANCE = 4
    NORMAL_DISTANCE = 3
    SMALL_DISTANCE = 2
    LOW_DISTANCE = 1
    EQUAL = 0
    def cabin_distance(u, v):
        _u, _v = u[0], v[0]
        if not isinstance(_u, Iterable) or not isinstance(_v, Iterable):
            return LONG_DISTANCE
        unums = list(map(int, sum(map(num_pat.findall, _u), [])))
        vnums = list(map(int, sum(map(num_pat.findall, _v), [])))
        ulets = list(sum(map(let_pat.findall, _u), []))
        vlets = list(sum(map(let_pat.findall, _v), []))
        if not(unums and vnums):
            if set(ulets).intersection(vlets):
                return EQUAL
            else:
                return MEDIUM_DISTANCE
        if u == v:
            return EQUAL
        if set(_u).intersection(set(_v)):
            return LOW_DISTANCE
        if not set(ulets).intersection(set(vlets)):
            return MEDIUM_DISTANCE
        else:
            for p in _u:
                for q in _v:
                    try:
                        pval = list(map(int, num_pat.findall(p)))[0]
                        qval = list(map(int, num_pat.findall(q)))[0]
                        if p[0] == q[0] and (abs(pval - qval) <= 2):
                            return SMALL_DISTANCE
                    except IndexError:
                        pass
            return NORMAL_DISTANCE
        return MEDIUM_DISTANCE
    distances = pdist(df.Cabin.apply(lambda x: x.split() if not isinstance(x, float) else x).values[:, np.newaxis], cabin_distance)
    df_ = df.copy()
    df_.loc[:, 'cabin_group'] = fcluster(complete(distances), SMALL_DISTANCE, criterion='distance')
    return df_


## Building pipelines

In [188]:
from mlpipes.pfunc import *

preprocessing_pipeline = (('add_groups', get_ticket_group, {}),
                          ('add_isalone', get_is_alone, {}),
                          ('add_titles', get_titles, {}),
                          ('convert_fares', discretize_faries, {'ngroups': 4}),
                          ('add_ticket_group', get_ticket_group, {}),
                          ('fill_embarked', fill_na_simple, {'colnames': ('Embarked',),
                                                             'methods': (lambda x: pd.Series(x).mode()[0],)}),
                          ('add_family_name', get_family_name, {}),
                          ('add_ages', estimate_age, {}),
                          ('add_cabin_groups', get_cabin_groups, {}),
                          ('drop_columns', drop_columns, {'colnames': ('Survived',
                                                                       'PassengerId',
                                                                       'SibSp',
                                                                       'Parch',
                                                                       'Ticket',
                                                                       'Fare',
                                                                       'family_name',
                                                                       'Name',
                                                                       
                                                                       )}),
                         )

def process(pipeline, data):
    data_ = data.copy()
    for name, func, kwargs in pipeline:
        print("=========== Step: %s ===========" % name)
        data_ = func(data_, **kwargs)
        print("=" * 40)
    return data_


# Preprocessing steps (feature engeneering)

In [189]:
processed  = process(preprocessing_pipeline, combined)

=========== Step: add_groups ===========
=========== Step: add_isalone ===========
=========== Step: add_titles ===========
=========== Step: convert_fares ===========
=========== Step: add_ticket_group ===========
=========== Step: fill_embarked ===========
=========== Step: add_family_name ===========
=========== Step: add_ages ===========
=========== Step: add_cabin_groups ===========
=========== Step: drop_columns ===========
